In [3]:
%pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import urllib.request

if not os.path.exists("test.txt"):
    url = ("https://raw.githubusercontent.com/Mengjie-Guo/LLM/master/GPT-small/test.txt")
    file_path = "test.txt"
    urllib.request.urlretrieve(url, file_path)

In [5]:
with open("test.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:19])

Total number of character: 20479
I HAD always though


In [33]:
text = "Good  night. Are you --ok?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(result)
result = [item.strip() for item in result if item.strip()]
print(result)

['Good', ' ', '', ' ', 'night', '.', '', ' ', 'Are', ' ', 'you', ' ', '', '--', 'ok', '?', '']
['Good', 'night', '.', 'Are', 'you', '--', 'ok', '?']


In [9]:
import re
pretext = re.split(r'([,.;?!"_()\']|\s|--)', raw_text)

In [11]:
pretext = [item.strip() for item in pretext if
item.strip()]

In [13]:
print(len(pretext))
print(pretext[:9])

4670
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap']


In [15]:
all_words=sorted(set(pretext))

In [17]:
word_cnt=len(all_words)
print(word_cnt)

1146


In [19]:
print(all_words[:10])

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';']


In [27]:
vocab = {token:index for index,token in enumerate(all_words)}
i=0
for key,val in vocab.items():
    i += 1
    print(key,":",val)
    if i > 10:
        break
    

! : 0
" : 1
' : 2
( : 3
) : 4
, : 5
-- : 6
. : 7
: : 8
; : 9
? : 10


In [73]:
class TokenizerV1:
    def __init__(self, vocab):
        self.str_int = vocab
        self.int_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        pretext = re.split(r'([,.;?!"_()\']|\s|--)', text)
        pretext = [item.strip() for item in pretext if item.strip()]
        indexs = [self.str_int[s] for s in pretext]
        return indexs

    def decode(self, indexs):
        text = " ".join(self.int_str[i] for i in indexs)
        # Replace spaces before the below punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [75]:
tokenizer = TokenizerV1(vocab)
text = """We're in the corner, Mr. Gisburn. "How're you?"""
indexs = tokenizer.encode(text)
print(indexs)

[109, 2, 830, 573, 1002, 292, 5, 67, 7, 38, 7, 1, 53, 2, 830, 1142, 10]


In [77]:
tokenizer.decode(indexs)

'We\' re in the corner, Mr. Gisburn." How\' re you?'

In [79]:
tokenizer.decode(tokenizer.encode(text))

'We\' re in the corner, Mr. Gisburn." How\' re you?'

In [83]:
# add special tokens unknown:"<|unk|>" and the end of a text "<|endoftext|>"
all_words.extend(["<|endoftext|>", "<|unk|>"])

In [87]:
vocab = {s:i for i,s in enumerate(all_words)}

In [91]:
print(len(all_words), len(vocab.items()))

1148 1148


In [117]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1143)
('your', 1144)
('yourself', 1145)
('<|endoftext|>', 1146)
('<|unk|>', 1147)


In [131]:
class TokenizerV2:
    def __init__(self, vocab):
        self.str_int = vocab
        self.int_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        pretext = re.split(r'([,.;?!"_()\']|\s|--)', text)
        #print(pretext)
        pretext = [item.strip() for item in pretext if item.strip()]
        pretext = [
            item if item in self.str_int
            else "<|unk|>" for item in pretext
        ]
        indexs = [self.str_int[s] for s in pretext]
        return indexs

    def decode(self, indexs):
        text = " ".join(self.int_str[i] for i in indexs)
        # Replace spaces before the below punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [133]:
tokenizer = TokenizerV2(vocab)

In [135]:
text1 = "Hi, Mr. Gisburn."
text2 = "Are you from?"
text = " <|endoftext|> ".join((text1,text2)) #space needed to avoid 
print(text)
indexs = tokenizer.encode(text)
print(indexs)
tokenizer.decode(indexs)


Hi, Mr. Gisburn. <|endoftext|> Are you from?
[1147, 5, 67, 7, 38, 7, 1146, 15, 1142, 480, 10]


'<|unk|>, Mr. Gisburn. <|endoftext|> Are you from?'

In [137]:
entext=tokenizer.encode(raw_text)

In [139]:
print(entext[:9])
print(len(entext))

[54, 45, 151, 1017, 58, 38, 829, 117, 258]
4670


In [155]:
for i in range(0, 5, 1):
    print(entext[i:i+2])

[54, 45]
[45, 151]
[151, 1017]
[1017, 58]
[58, 38]


In [153]:
#next-word prediction
ensample = entext[:50]
context_size = 5
for i in range(1, context_size+1, 2):
    context = ensample[:i]
    predicted = ensample[i]
    print(i, tokenizer.decode(context), "->", tokenizer.decode([predicted]))

1 I -> HAD
3 I HAD always -> thought
5 I HAD always thought Jack -> Gisburn


In [151]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.7.0


In [195]:
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, maxlen, stride):
        self.input_ids = []
        self.target_ids = []
        
        tokenids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})
        print(tokenids[:50])
        assert len(tokenids) > maxlen, "Input token length can't less than maxlen"

        #sliding window to chunk text
        for i in range(0, len(tokenids) - maxlen, stride):
            input = tokenids[i:i+maxlen]
            target = tokenids[i+1:i+1+maxlen]
            self.input_ids.append(input)
            self.target_ids.append(target)
    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
    

In [175]:
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 kB/s eta 0:00:0000:0100:03
Note: you may need to restart the kernel to use updated packages.


In [197]:
import tiktoken
def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    print(dataset.__len__())

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        drop_last=drop_last)

    return dataloader
    

In [201]:
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False, drop_last=True, num_workers=0)

dataiter=iter(dataloader)
inputs, targets = next(dataiter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]
1286
Inputs:
 [tensor([   40,  1807, 10899, 15632,   922,   568,  1049,   284]), tensor([ 367, 3619, 2138,  438, 5891,  340, 5975, 3285]), tensor([2885,  402,  257, 2016, 1576,  373,  284,  326]), tensor([1464,  271, 7026,  257,  438,  645,  502,   11])]

Targets:
 [tensor([ 367, 3619, 2138,  438, 5891,  340, 5975, 3285]), tensor([2885,  402,  257, 2016, 1576,  373,  284,  326]), tensor([1464,  271, 7026,  257,  438,  645,  502,   11]), tensor([ 1807, 10899, 15632,   922,   568,  1049,   284,   287])]


In [207]:
inputids = [4,6,3,1,5]
# small vocab contains only 7 words
vocabsize=7
outputdim=4
torch.manual_seed(123)
embeddinglayer=torch.nn.Embedding(vocabsize, outputdim)

In [209]:
print(embeddinglayer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880],
        [ 0.3486,  0.6603, -0.2196, -0.3792],
        [ 0.7671, -1.1925,  0.6984, -1.4097],
        [ 1.8960, -0.1750,  1.3689, -1.6033],
        [-1.3250,  0.1784, -2.1338,  1.0524],
        [ 0.9985,  0.2212,  1.8319, -0.3378],
        [ 0.8805,  1.5542,  0.6266, -0.1755]], requires_grad=True)


In [211]:
print(embeddinglayer(torch.tensor(5)))

tensor([ 0.9985,  0.2212,  1.8319, -0.3378], grad_fn=<EmbeddingBackward0>)


In [217]:
print(embeddinglayer(torch.tensor(inputids)))

tensor([[-1.3250,  0.1784, -2.1338,  1.0524],
        [ 0.8805,  1.5542,  0.6266, -0.1755],
        [ 1.8960, -0.1750,  1.3689, -1.6033],
        [ 0.3486,  0.6603, -0.2196, -0.3792],
        [ 0.9985,  0.2212,  1.8319, -0.3378]], grad_fn=<EmbeddingBackward0>)


In [219]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)